<a href="https://www.kaggle.com/code/toptm123/smart-closet?scriptVersionId=283169802" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from typing import Any, Dict
import google.genai
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.agent_tool import AgentTool
from google.genai import types
from google.adk.runners import InMemoryRunner
import sqlite3
import pandas as pd
import uuid
from datetime import datetime, timedelta
import pytz

print("✅ ADK components imported successfully.")

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
try:
    Alter = UserSecretsClient().get_secret("Alter")
    os.environ["Alter"] = Alter
    
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'Alter' to your Kaggle secrets. Details: {e}"
    )


In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    project_name = UserSecretsClient().get_secret("project_name")
    os.environ["project_name"] = project_name
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error. Details: {e}"
    )

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)
print("retry configs set")

Installing packages

In [ ]:
!pip install earthengine-api
!pip install geonamescache

In [ ]:
import ee
import geonamescache

Initializing earth engine

In [ ]:
ee.Authenticate()

# Initialize the library.
ee.Initialize(project=project_name)
gfs = ee.ImageCollection('NOAA/GFS0P25')

# Closet setup

In [ ]:


db_url = "smart_closet_memory.db"

# connect
con = sqlite3.connect(db_url)

# import the sample closet items from CSV

df = pd.read_csv("/kaggle/input/d/toptm123/closet/closet.csv",)

# write entire DataFrame into table "Closet"
df.to_sql(
    name="Closet",
    con=con,
    if_exists="replace",   # overwrite table if it exists
    index=False)            

cur = con.cursor()

cur.execute("ALTER TABLE Closet ADD COLUMN avail BINARY DEFAULT 1") # add availability values

for row in cur.execute("SELECT * FROM Closet"): #view closet items
    print(row)

con.close()

# Closet Manager
An optional agent that maintains the closet database

## Closet Manger helper functions

In [ ]:
def update_availability(item:str, avail:int,db_url:str = "smart_closet_memory.db"):
    """Args:
    item: string of item name
    db_url: string of database location
    avail: int of availability. 0 is unavailable, 1 is available
    """
    with sqlite3.connect("smart_closet_memory.db") as con:
        cur = con.cursor()
        cur.execute('UPDATE Closet SET avail = ? WHERE "Item Name" = ?', (avail,item,))

def add_item(item:str,Type:str, season:str, formality:int, db_url:str = "smart_closet_memory.db"):
    """Args:
    item: string of item name
    db_url: string of database location
    season: the season that item is suitable for 
    Type: item type(ex. Top, Bottom, One-piece)
    formality: 0 is casual, 4 is completely formal
    """
    con = sqlite3.connect(db_url)
    cur = con.cursor()
    cur.execute("""
    INSERT INTO Closet ("Item Name", Type, "Formal Level", "Season Focus",avail)
    VALUES (?, ?, ?,?,?)
    """, (item, Type, formality, season,1))

    con.commit()
    con.close()

def drop_item(item:str, db_url:str = "smart_closet_memory.db"):
    """Args:
    item: string of item name
    db_url: string of database location
    
    """
    with sqlite3.connect("smart_closet_memory.db") as con:
        cur = con.cursor()
        cur.execute('DELETE FROM Closet WHERE "Item Name" = ?',(item,))

In [ ]:
manager_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="closet_manager",
    instruction=f"""You are a closet manager agent. You maintain the closet database based on user's request.
    You are allowed to do the following actions:
    1. When user tries to make something available/unavailable, use `update_availability` tool to update the availability of individual item
    2. When user tries add something, use `add_item` to add individual item into closet. Infer missing arguments based on item name, dont ask.
    3. When user tries to remove/delete something, use `drop_item`""",
    tools=[FunctionTool(update_availability),FunctionTool(add_item), FunctionTool(drop_item)]
)

In [ ]:
# Demo: add an item
runner = InMemoryRunner(agent=manager_agent)
response = await runner.run_debug(
    "add purple T-shirt"
)


In [ ]:
# Demo: view the database
db_url = "smart_closet_memory.db"

# connect
con = sqlite3.connect(db_url)
cur=con.cursor()
for row in cur.execute("SELECT * FROM Closet"):
    print(row)

# Get Weather
A tool that returns weather data such as temperature, wind, and precipitation on a day at a place.

## Get weather helper functions

In [ ]:
def next_day(x:datetime) -> datetime:
    
    #d=datetime.fromisoformat(x)
    delta=timedelta(days=1)
    next_day=x+delta
    return next_day#.strftime("%Y-%m-%d")

def get_most_recent(col:ee.imagecollection.ImageCollection) -> datetime:
    latest_image = col.sort('system:time_start', False).first()
    latest_date = ee.Date(latest_image.get('system:time_start')).getInfo()
    x=datetime.fromtimestamp(latest_date.get('value')/1000) 
    print("Most recent image date:", x)
    return x


In [ ]:
def get_weather(date:str, longitude:float,latitude:float,
                timezone:str='Australia/Sydney'):
    """Args:
    date: string in iso format (ex.2025-02-01)
    """
    coords=(longitude,latitude)
    local=pytz.timezone(timezone)
    
    d=datetime.fromisoformat(date) #this is local time
    local_date=local.localize(d)
    utc_date=local_date.astimezone(pytz.utc) #convert local time to utc
    delta=timedelta(days=16)
    recent_run= pytz.utc.localize(get_most_recent(gfs))#this is utc time
    forecast=recent_run+delta
    #print(utc_date)
    if utc_date>forecast: #if date is after available prediction, we can't extract its weather forecast
        return False
    #return True
    day2=next_day(recent_run)
    gfs_run = gfs.filterDate(recent_run, day2)
    point = ee.Geometry.Point(coords)
    m = gfs_run \
      .filter(ee.Filter.inList('forecast_hours', list(range(0,385,1)))) \
      .select(['temperature_2m_above_ground',
               'specific_humidity_2m_above_ground',
               'gust',
              "precipitation_rate",
              ]) \
      .getRegion(point, scale=25000) \
      .getInfo()
    #print(len(m))
    #print(m)
    df=pd.DataFrame(m[1:],columns=m[0])
    #print(df.time[0])
    #print(df.time.tail(1))
    ts = df[df.time==df.iloc[-1].time].copy() #get the lastest run


    f_strip =  ts['id'].str[-3:].fillna("00").map(int) #strip the hour from id
#print(f_strip)
    f_time= pd.to_timedelta(f_strip,unit='hours')#parse digits into time in hours
    ts.loc[:,"time_utc"]=pd.to_datetime(df['time'],unit='ms')+f_time #the predicted time is run time+time lapse


    ts=ts.drop(columns=['id','latitude','longitude','time'])

    ts=ts.rename(columns={"temperature_2m_above_ground":"predicted_temp",
                         "specific_humidity_2m_above_ground":"humidity",
                          "precipitation_rate":"precipitation"
                         })
    ts['precipitation']=ts['precipitation']*3600 # convert to hourly 

    time_local=pd.DataFrame(ts.time_utc).map(lambda x: x.tz_localize(pytz.utc).tz_convert(local))
    ts=ts.assign(time_local=time_local)
    #print(ts)
    dti = pd.date_range(local_date,next_day(local_date), freq="h",tz=local)
    sub=ts[ts.time_local.isin(dti)].drop(columns=["time_utc"])
    return sub.to_dict(orient="records")
                        
#print(get_weather("2025-12-05",151.2,33.9)) 


# Response parsing helpers

In [ ]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    #print("scanning for events")
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    approval_id= part.function_call.id
                    #pid=part.function_call.args['originalFunctionCall']["id"]
                    #print(part.function_call.args["toolConfirmation"])
                    return {
                        "approval_id": approval_id,
                        "invocation_id": event.invocation_id,
                        "payload":part.function_call.args["toolConfirmation"]["payload"],
                    }
    return None

def create_approval_response(approval_info, approved:str):
    """Create approval response message."""
    #print("writing approval message")
    #print("approval status: "+approved)
    z=approval_info["payload"]
    z["confirmed"]=approved
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"payload":z},#user;s response
        
    )
    pp=types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )
    #print(pp)
    return pp



# Sample tool
A tool that returns coordinates and timezone of a location.

In [ ]:


#import geonamescache
def sample_tool(x:str,tool_context: ToolContext):
    """returns coordinates and timezone of x"""
    #n=len(x)
    gc = geonamescache.GeonamesCache()
    l = gc.search_cities(x,case_sensitive=False, contains_search=False)
    results= [{"name": i['name'], "latitude":i['latitude'],"longitude":i['longitude'],"timezone":i['timezone']} for i in l]
    
    if len(results)==1:
        tup=(results[0]['longitude'],results[0]['latitude'])
        return {"status":"confirmed",
                "message":"The city is identified, return the coordinates",
                #"coordinates":tup,
                "timezone": results[0]['timezone'],
                'longitude':tup[0],
               "latitude":tup[1]}
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"{len(results)} cities found. Please select the desired city from results",
            payload={"results": results},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"""{len(results)} cities found. List the cities from {results} in the form of [Name, Province/State, Country]
            Ask user to select the desired city from the list of results by numbering""",
            #"results": results
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.payload:
        #print(f"\n\n tool context {tool_context.tool_confirmation}")
        response=int(tool_context.tool_confirmation.payload["confirmed"])
        #print(f"Context: {response}")
        #print(results)
        if response<len(results):
            tup=(results[response-1]['longitude'],results[response-1]['latitude'])
            timezone=results[response-1]['timezone']
            return {
                "status": "confirmed",
                "message": f"City selected",
                'longitude':tup[0],
               "latitude":tup[1],
                "timezone":timezone
            }
        else:
            return {
                "status": "rejected",
                "message": f"Failed",
            }


# City lookup Tool
Actually an agent tool. It looks up the location from the prompt and returns its coordinates and timezone. If there are multiple cities with the same name, it asks for user confirmation. 

In [ ]:

session_service = InMemorySessionService()

prompt =f"""
Use sample_tool to look up the city's coordinates
If the status is pending, ask the user to select one option from the list the cities in the form of [Name, Province/State, Country].
If the status is confirmed, return the coordinates of the city and its timezone as a bullet list.
"""

city_lookup_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="city_lookup_agent",
    description="Retrives coordinate based on city",  # Description of the agent's purpose
    instruction=prompt,
    tools=[FunctionTool(func=sample_tool)]
)

lookup_app = App(
    name="lookup_app",
    root_agent=city_lookup_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)
lookup_runner = Runner(
    app=lookup_app,
    session_service=session_service
)


async def city_lookup(city: str,auto_approve:str="1"):
    """Input: city name, auto_approve
    Output: a dictionary containing coordinates and timezone."""
    print(f"\n{'='*60}")
    print(f"User > {city}\n")

    session_id = uuid.uuid4().hex[:8]
    
    await session_service.create_session(
        app_name="lookup_app",
        user_id="test_user",
        session_id=session_id
    )
    
    query = types.Content(
        role="user",
        parts=[types.Part(text=city)]
    )

    events = []
    async for event in lookup_runner.run_async(
        user_id="test_user",
        session_id=session_id,
        new_message=query
    ):
        events.append(event)
        #print(f"\n\n{event}")
    approval_info = check_for_approval(events)
   
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        results_1=approval_info['payload']['results']
        client = google.genai.Client(vertexai=False, api_key=Alter)
        
        agent_reply = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=f"""Turn the dictionary {results_1} into a numbered list of [city name, province/state, country].
                You should provide your reply like `Please select one option from the following: [the list]""",
            )
        print(agent_reply.text)
        print(f"🤔 Human Decision: {auto_approve}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        
        async for event in lookup_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
            approval_info, auto_approve
        ),  # Send human decision here
        invocation_id=approval_info[
            "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            #print("##### return to agent")
            if event.content and event.content.parts:
                for part in event.content.parts:
                    #print(events[-1])
                    if part.text:
                        print(f"Agent > {part.text}")
                    if part.function_response:
                        d={'longitude':part.function_response.response['longitude'],
                           'latutude':part.function_response.response['latitude'],
                           'timezone':part.function_response.response['timezone']}
        return d
                        

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)
        event = events[-3]
        for part in event.content.parts:
                    if part.function_response:
                        d={'longitude':part.function_response.response['longitude'],
                           'latutude':part.function_response.response['latitude'],
                           'timezone':part.function_response.response['timezone']}
        return d
    #print(events[-1])       

print("✅ Workflow function ready")


# Stylist agent
It queries clothing items and uses weather data to produce outfit plans.

In [ ]:
#import sqlite3

def query_to_dict(db_url:str="smart_closet_memory.db", query:str="SELECT * FROM Closet"):
    """Execute SQL query and return rows as a list of dictionaries."""
    con = sqlite3.connect(db_url)
    cur = con.cursor()

    # Get column names
    cur.execute(query)
    rows = cur.fetchall()
    col_names = [desc[0] for desc in cur.description]

    con.close()

    # Convert each row to a dict
    result = [dict(zip(col_names, row)) for row in rows]
    return result


In [ ]:
prompt_1 =f"""
You are a personal stylist agent.

Inputs:
1. Weather data containing the target date (temperature, precipitation, gust)


Task:
- Use `query_to_dict` tool to get the items
- Recommend a complete outfit appropriate for the occasion based on closet items and weather data.
- Consider:
    * Occasion formality: `Formal Level` of 0 is casual, 4 is completely formal
    * Weather conditions (rain, temperature)
    * Seasonal appropriateness
    * Matching items (colors, style)
- Return recommendations as a structured list of items (top & bottom or one-piece, outerwear if needed). 
Keep your response concise.
"""


planning_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="planning_agent",
    instruction=prompt_1,
    tools=[FunctionTool(query_to_dict)]
    
)

planning_runner = Runner(
    agent=planning_agent,
    app_name="outfit_planner_app",
    session_service=session_service
)


In [ ]:
def timenow():
    return datetime.utcnow()

# Root agent
The master agent that
- confirms location
- gathers weather data
- returns outfit plans

In [ ]:

root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    name="manager",
    instruction=f"""You are a manager agent. Your goal is to plan the user's outfit based on weather and occasion.
    If exact date is not given, use `timenow` tool to get current time and infer the date in iso format.
    1. You MUST use `city_lookup` tool to find the coordinates and timezone
    2. You MUST use `get_weather` tool to extract weather data of the target date. You should mainly focus on weather from 7am to 9pm, but remind the user if extreme weather takes place at night time. 
    3. You MUST use `planner_agent` tool to compose the outfit plan""",
    tools=[FunctionTool(get_weather),FunctionTool(city_lookup),AgentTool(planning_agent),FunctionTool(timenow)]
)
my_app = App(
    name="my_app",
    root_agent=root_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)
my_runner = Runner(
    app=my_app,
    session_service=session_service
)


# Smart Closet Outfit Planning
An use case demonstration

In [ ]:

session_id = uuid.uuid4().hex  # full UUID, unique

# STEP 1: Prepare the user message
text = "I'm going to a concert in Syndey on Dec 3"
query = types.Content(
    role="user",
    parts=[types.Part(text=text)]
)

await session_service.create_session(
        app_name="my_app",
        user_id="test_user",
        session_id=session_id
    )
events_1 = []
async for event in my_runner.run_async(
    user_id="test_user",
    session_id=session_id,
    new_message=query
):
    events_1.append(event)
    #print(event)

# STEP 3: Examine results
print_agent_response(events_1)